### 0. 모델 로드 및 전처리 등 실험과 상관 없는 내용

In [1]:
import torch
import torch.nn as nn
from torchvision import models,datasets
from me_models import MultiExitViT
from me_ResNet import MultiExitResNet
from Dloaders import Dloaders
####################################################################
IMG_SIZE = 224
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_name = {'cifar10':datasets.CIFAR10, 'cifar100':datasets.CIFAR100,'imagenet':None}
dataset_outdim = {'cifar10':10, 'cifar100':100,'imagenet':1000}
##############################################################
################ 0. Hyperparameters ##########################
##############################################################
batch_size = 1024
data_choice='imagenet'
model_choice='vit' #vit or resnet
mevit_isload=True
me_ptd_path=f"models/{model_choice}/{data_choice}/integrated_ee.pth"
backbone_path=f'models/{model_choice}/{data_choice}/{model_choice}_{data_choice}_backbone.pth'

file_path = f'cache_result_{model_choice}_{data_choice}.pt'

ee_list=[0,1,2,3,4,5,6,7,8,9]#exit list ex) [0,1,2,3,4,5,6,7,8,9]
exit_loss_weights=[1,1,1,1,1,1,1,1,1,1,1]#exit마다 가중치
exit_num=11
##############################################################
dloaders=Dloaders(data_choice=data_choice,batch_size=batch_size,IMG_SIZE=IMG_SIZE)
train_loader,test_loader = dloaders.get_loaders()

# Load the pretrained ViT model from the saved file
if model_choice == 'vit':
    # Load the pretrained ViT model from the saved file
    ptd_model = models.vit_b_16(weights=None)
    ptd_model.heads.head = nn.Linear(ptd_model.heads.head.in_features, dataset_outdim[data_choice])  # Ensure output matches the number of classes
    # Load model weights
    new_model = MultiExitViT(base_model=ptd_model,num_classes=dataset_outdim[data_choice])
elif model_choice == 'resnet':
    # Load the pretrained ResNet model from the saved file
    ptd_model = models.resnet101()
    ptd_model.fc = nn.Linear(ptd_model.fc.in_features, dataset_outdim[data_choice])
    # Load model weights
    new_model = MultiExitResNet(base_model=ptd_model,num_classes=dataset_outdim[data_choice])    

new_model.load_state_dict(torch.load(me_ptd_path))
new_model.to(device)

# Assume a pretrained model (replace with your own model)
new_model.load_state_dict(torch.load(me_ptd_path))  # Load your trained weights

Using the latest cached version of the dataset since imagenet-1k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at data/imagenet/imagenet-1k/default/1.0.0/07900defe1ccf3404ea7e5e876a64ca41192f6c07406044771544ef1505831e8 (last modified on Fri Nov 15 11:12:13 2024).


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

Using the latest cached version of the dataset since imagenet-1k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at data/imagenet/imagenet-1k/default/1.0.0/07900defe1ccf3404ea7e5e876a64ca41192f6c07406044771544ef1505831e8 (last modified on Fri Nov 15 11:12:13 2024).


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

/tmp/ipykernel_54779/2455132039.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model.load_state_dict(torch.load(me_ptd_path))
/tmp/ipykernel_54779/2455132039.py:49

<All keys matched successfully>

### 1. MEVIT의 각 출구들에서의 Precision을 확인

In [2]:
resnet_cifar10_acc = [64.47, 72.24, 81.45, 84.25, 86.01, 90.24, 92.6, 94.31, 95.77, 95.08, 97.52]
resnet_cifar100_acc = [37.8, 44.83, 60.08, 67.39, 72.1, 74.34, 77.32, 79.9, 80.94, 83.05, 86.24]
resnet_imagenet_acc = [28.278, 35.742, 51.206, 54.662, 60.53, 63.732, 67.158, 70.394, 73.672, 79.374, 80.888]
vit_cifar10_acc = [72.76, 80.35, 85.8, 89.42, 91.98, 93.12, 94.94, 96.44, 96.95, 97.35, 97.63]
vit_cifar100_acc = [48.21, 59.63, 67.93, 73.31, 77.9, 80.8, 83.71, 85.33, 86.61, 87.04, 87.75]
vit_imagenet_acc = [34.76, 42.65, 51.82, 57.76, 62.19, 65.5, 69.29, 72.3, 75.3, 77.6, 81.06]

resnet_cifar10_flops = [1412860928, 2076990464, 2890800128, 3550413824, 4210027520, 4869641216, 5529254912, 6188868608, 6848482304, 7661519360, 7880637952]
resnet_cifar100_flops = [1412907008, 2077036544, 2890892288, 3550505984, 4210119680, 4869733376, 5529347072, 6188960768, 6848574464, 7661703680, 7880822272]
resnet_imagenet_flops = [1413367808, 2077497344, 2891813888, 3551427584, 4211041280, 4870654976, 5530268672, 6189882368, 6849496064, 7663546880, 7882665472]
vit_cifar10_flops = [2908083456, 4303940352, 5699797248, 7095654144, 8491511040, 9887367936, 11283224832, 12679081728, 14074938624, 15470795520, 16866652416]
vit_cifar100_flops = [2908152576, 4304009472, 5699866368, 7095723264, 8491580160, 9887437056, 11283293952, 12679150848, 14075007744, 15470864640, 16866721536]
vit_imagenet_flops= [2908843776, 4304700672, 5700557568, 7096414464, 8492271360, 9888128256, 11283985152, 12679842048, 14075698944, 15471555840, 16867412736]

### 2. MEVIT의 각 출구들까지의 FLOPs 확인

In [3]:
'''from torchinfo import summary    #wrong result in multi-head attention module in vit
tmp=[]
for i in range(exit_num):
    new_model.set_each_ee_test_mode(i)
    #summary(new_model,input_size= (1, 3, IMG_SIZE, IMG_SIZE))
    tmp.append(summary(new_model,input_size= (1, 3, IMG_SIZE, IMG_SIZE)).total_mult_adds)
tmp''' 
from fvcore.nn import FlopCountAnalysis

tmp = []
for i in range(exit_num):
    new_model.set_each_ee_test_mode(i)
    x = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
    flops = FlopCountAnalysis(new_model, x)
    tmp.append(flops.total())
print(', '.join(map(str, tmp)))

Unsupported operator aten::mul encountered 9 time(s)
Unsupported operator aten::add encountered 5 time(s)
Unsupported operator aten::div encountered 2 time(s)
Unsupported operator aten::unflatten encountered 2 time(s)
Unsupported operator aten::scaled_dot_product_attention encountered 2 time(s)
Unsupported operator aten::gelu encountered 2 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
base_model.encoder.layers.encoder_layer_0.self_attention.out_proj, base_model.encoder.layers.encoder_layer_1, base_model.encoder.layers.encoder_layer_1.dropout, base_model.encoder.layers.encoder_layer_1.ln_1, base_model.encoder.layers.encoder_layer_1.ln_2, base_model.encoder.layers.encoder_layer_1.mlp, base_model.encoder.layer

2908843776, 4304700672, 5700557568, 7096414464, 8492271360, 9888128256, 11283985152, 12679842048, 14075698944, 15471555840, 16867412736


In [4]:
# ResNet101 FLOPs: 16866721536
# 7799667684(resnet101) VS 16866721536(ViT-b/16)
from torchinfo import summary
pretrained_resnet101 = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
pretrained_resnet101.fc = nn.Linear(pretrained_resnet101.fc.in_features, dataset_outdim[data_choice])
summary(pretrained_resnet101,input_size= (1, 3, IMG_SIZE, IMG_SIZE)).total_mult_adds

7801511784

In [5]:
print(16866721536 / 7799667684 * 100,'%') #ViT-b/16의 flops가 resnet101의 216.24%이다.. 즉 2배이다.

216.24923290770295 %


In [6]:
# 만약 앙상블을 한다면 총 연산량은 단일출구에 비해 1.8배가 될 것이다.
print(30832862976 / 16866721536 * 100,'%')

182.80294075046498 %
